# **Assignment 4**

The objective of the assignment is to fit a provided set of 4 data traces against different distributions using the *method of moments* and the *maximum likelihood method* and compare against the data on a plot.

## Initialization
First of all, we need to:
- tell ipython how to treat the generated plots
- import the necessary libraries
- read the data sets from the provided csv file.

In [16]:
%matplotlib tk
import csv
import numpy as np
from scipy.optimize import root
from scipy.optimize import minimize
from matplotlib import pyplot as plt

data = csv.reader(open("./Assignment and Dataset/Traces.csv"))
data = [[float(x) for x in row] for row in data]
data = np.transpose(data)

## Method of moments
Let's fit the data against some distributions by using the method of moments first.
In particular, we will try to fit the data against:
- a uniform distribution
- an exponential distribution
- a two stages hyper-exponential distribution
- a two stages hypo-exponential distribution

### Finding the moments
We need to find the first three moments of each data trace to be used to estimate the parameters.

In [2]:
m1 = np.mean(data, 1)
m2 = np.mean([[x**2 for x in row] for row in data], 1)
m3 = np.mean([[x**3 for x in row] for row in data], 1)

print("{!s: <18}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <18}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("first moment",m1[0],m1[1],m1[2],m1[3]))
print("{!s: <18}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("second moment",m2[0],m2[1],m2[2],m2[3]))
print("{!s: <18}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("third moment",m3[0],m3[1],m3[2],m3[3]))

                  Dataset 1      Dataset 2      Dataset 3      Dataset 4      

first moment      7.4913         6.9630         7.7766         7.7348         
second moment     58.2225        102.1096       205.2542       94.3765        
third moment      467.7512       2316.8227      10849.6130     1609.7027      


### Uniform distribution
Find the parameters of a uniform distribution that better fit the provided data traces.

In [3]:
def mm_unif_pars(moment1, moment2):
    a = moment1 - 1/2*(np.sqrt(12*(moment2-moment1**2)))
    b = moment1 + 1/2*(np.sqrt(12*(moment2-moment1**2)))
    return [a, b]

mm_Unif = []
for i in range(0,4):
    mm_Unif.append(mm_unif_pars(m1[i], m2[i]))


print("{!s: <6}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <6}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("a",mm_Unif[0][0],mm_Unif[1][0],mm_Unif[2][0],mm_Unif[3][0]))
print("{!s: <6}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("b",mm_Unif[0][1],mm_Unif[1][1],mm_Unif[2][1],mm_Unif[3][1]))

      Dataset 1      Dataset 2      Dataset 3      Dataset 4      

a     4.9795         -5.7208        -13.0640       -2.4459        
b     10.0031        19.6468        28.6173        17.9156        


### Exponential distribution
Find the parameters of an exponential distribution that better fit the provided data traces.

In [4]:
def mm_exp_pars(moment1):
    l = 1/moment1
    return l

mm_Exp = []
for i in range(0,4):
    mm_Exp.append(mm_exp_pars(m1[i]))
print("{!s: <10}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <10}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda",mm_Exp[0],mm_Exp[1] ,mm_Exp[2] ,mm_Exp[3]))

          Dataset 1      Dataset 2      Dataset 3      Dataset 4      

lambda    0.1335         0.1436         0.1286         0.1293         


### Hyper-exponential distribution
Find the parameters of a hyper-exponential distribution that better fit the provided data traces.

In [5]:
def mm_hyper_pars(moment1, moment2, moment3, start):
    def hyper_eqs(par):
        lambda1, lambda2, p = par
        if lambda1<0:
            return (100000 + abs(lambda1), 100000 + abs(lambda1), 100000 + abs(lambda1))
        if lambda2<0:
            return (100000 + abs(lambda2), 100000 + abs(lambda2), 100000 + abs(lambda2))
        if p>1:
            return (100000 + abs(p), 100000 + abs(p), 100000 + abs(p))
        if p<0:
            return (100000 + abs(p), 100000 + abs(p), 100000 + abs(p))

        return[((p/lambda1 + (1-p)/lambda2))/moment1-1,
               (2*(p/(lambda1**2) + (1-p)/(lambda2**2)))/moment2-1,
               (6*(p/(lambda1**3) + (1-p)/(lambda2**3)))/moment3-1]

    sol = root(hyper_eqs, start, method='lm')
    return sol.x

mm_Hyper = []
for i in range(0,4):
#    if i != 2:
#       mm_Hyper.append(mm_hyper_pars(m1[i], m2[i], m3[i], [0.5, 0.5, 0.5]))
#    else:
#        mm_Hyper.append(mm_hyper_pars(m1[i], m2[i], m3[i], [0.5, 0.5, 0.0]))
        mm_Hyper.append(mm_hyper_pars(m1[i], m2[i], m3[i], [0.5, 0.5, 0.5]))

print("{!s: <12}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <12}{:<15.9f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 1",mm_Hyper[0][0],mm_Hyper[1][0] ,mm_Hyper[2][0] ,mm_Hyper[3][0]))
print("{!s: <12}{:<15.9f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 2",mm_Hyper[0][1],mm_Hyper[1][1] ,mm_Hyper[2][1] ,mm_Hyper[3][1]))
print("{!s: <12}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("p",mm_Hyper[0][2],mm_Hyper[1][2] ,mm_Hyper[2][2] ,mm_Hyper[3][2]))

            Dataset 1      Dataset 2      Dataset 3      Dataset 4      

lambda 1    0.221210209    0.1274         0.0507         0.1511         
lambda 2    0.221210216    0.2459         0.2355         0.1511         
p           0.5002         0.7655         0.2281         0.5020         


*Note: I printed some more digits for the first dataset because I wanted to show that the two lambdas are not the same (in which case the hyper exponential would have been a simple exponential, but with a different parameter than the one obtained for the exponential).*

### Hypo-exponential distribution
Find the parameters of a hypo-exponential distribution that better fit the provided data traces.

In [6]:
def mm_hypo_pars(moment1, moment2, start):
    def hypo_eqs(par):
        lambda1, lambda2 = par
        return((1/lambda1 + 1/lambda2)/moment1 - 1,
               (1/lambda1**2 + 1/lambda2**2 + 1/(lambda1*lambda2))*2/moment2 - 1)
    sol = root(hypo_eqs, start, method='lm')
    return sol.x

mm_Hypo = []
for i in range(0,4):
    mm_Hypo.append(mm_hypo_pars(m1[i],  m2[i],  [0.5,  0.5]))
    
print("{!s: <12}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <12}{:<15.9f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 1",mm_Hypo[0][0],mm_Hypo[1][0] ,mm_Hypo[2][0] ,mm_Hypo[3][0]))
print("{!s: <12}{:<15.9f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 2",mm_Hypo[0][1],mm_Hypo[1][1] ,mm_Hypo[2][1] ,mm_Hypo[3][1]))

            Dataset 1      Dataset 2      Dataset 3      Dataset 4      

lambda 1    0.312041759    0.1407         0.1067         0.1855         
lambda 2    0.312038686    11380425.6737  6834624.7702   0.4264         


## Maximum likelihood method
Let's now fit the data against some distributions by using the maximum likelihood method.
In particular, we will try to fit the data against:
- an exponential distribution
- a two stages hyper-exponential distribution
- a two stages hypo-exponential distribution

### Exponential distribution
Find the parameters of an exponential distribution that better fit the provided data traces.

In [7]:
def ml_exp_pars(start, values):

    def exp_eqs(lamb):
        return (-len(values)*np.log(lamb)) + (lamb*np.sum(values))

    sol = minimize(exp_eqs, x0=start, bounds=[(0, np.Inf)])
    return sol.x

ml_Exp = []
for i in range(0,4):
    ml_Exp.append(ml_exp_pars(mm_Exp[i], data[i]))

print("{!s: <10}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <10}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda",ml_Exp[0][0],ml_Exp[1][0] ,ml_Exp[2][0] ,ml_Exp[3][0]))


          Dataset 1      Dataset 2      Dataset 3      Dataset 4      

lambda    0.1335         0.1436         0.1286         0.1293         


### Hyper-exponential distribution
Find the parameters of a hyper-exponential distribution that better fit the provided data traces.

In [8]:
def ml_hyper_pars(start, values):
    def hyper_eqs(par):
        [l1, l2, p] = par
        alt_values = [np.log(p*l1*np.exp(-l1*x)+(1-p)*l2*np.exp(-l2*x)) for x in values]
        return -np.sum(alt_values)
    sol = minimize(hyper_eqs, x0=start, bounds=[(0.01, np.Inf), (0.01, np.Inf), (0.01,1)])
    return sol.x

ml_Hyper = []
for i in range(0,4):
    ml_Hyper.append(ml_hyper_pars(mm_Hyper[i], data[i]))
    
print("{!s: <10}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <10}{:<15.8f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 1",ml_Hyper[0][0],ml_Hyper[1][0] ,ml_Hyper[2][0] ,ml_Hyper[3][0]))
print("{!s: <10}{:<15.8f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 2",ml_Hyper[0][1],ml_Hyper[1][1] ,ml_Hyper[2][1] ,ml_Hyper[3][1]))
print("{!s: <10}{:<15.4f}{:<15.4f}{:<15.4f}{:<15.4f}".format("p",ml_Hyper[0][2],ml_Hyper[1][2] ,ml_Hyper[2][2] ,ml_Hyper[3][2]))

          Dataset 1      Dataset 2      Dataset 3      Dataset 4      

lambda 1  0.13348830     0.1148         0.0487         0.1293         
lambda 2  0.13348831     0.1858         0.2233         0.1293         
p         0.5003         0.4746         0.2055         0.5020         


### Hypo-exponential distribution
Find the parameters of a hypo-exponential distribution that better fit the provided data traces.

In [9]:
def ml_hypo_pars(start, values):
    def hypo_eqs(par):
        [l1, l2] = par
        if l1 == l2: return 100000
        alt_values = [np.log(l1*l2/(l1-l2)*(np.exp(-l2*x)-np.exp(-l1*x))) for x in values]
        return -np.sum(alt_values)
    sol = minimize(hypo_eqs, x0=start, bounds=[(0.01, np.Inf), (0.01, np.Inf)])
    return sol.x

ml_Hypo = []
for i in range(0,4):
    ml_Hypo.append(ml_hypo_pars(mm_Hypo[i], data[i]))
    
print("{!s: <10}{!s:<15}{!s:<15}{!s:<15}{!s:<15}\n".format("","Dataset 1","Dataset 2","Dataset 3","Dataset 4"))    
print("{!s: <10}{:<15.8f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 1",ml_Hypo[0][0],ml_Hypo[1][0] ,ml_Hypo[2][0] ,ml_Hypo[3][0]))
print("{!s: <10}{:<15.8f}{:<15.4f}{:<15.4f}{:<15.4f}".format("lambda 2",ml_Hypo[0][1],ml_Hypo[1][1] ,ml_Hypo[2][1] ,ml_Hypo[3][1]))

          Dataset 1      Dataset 2      Dataset 3      Dataset 4      

lambda 1  0.26697725     0.1436         0.1286         0.1874         
lambda 2  0.26697431     11380425.6737  6834624.7702   0.4168         


## Plots
To visually compare how well the distributions are fit to each datatrace, we plot the former and the latter.
Since the visualization would have been impossible with all the plots crammed in a single graph, we decided to split them on 4 different plots for each dataset.

In [10]:
def gen_plot_old(i):
    
    fig, ((hyper, hypo), (exp, unif)) = plt.subplots(2, 2)
    fig.suptitle('Dataset '+str(i+1))

    ydata = [*range(0,len(data[i]))]
    ydata = np.divide(ydata,len(data[i]))
    xdata = np.sort(data[i])

    xlin  = np.linspace(start=0, stop=50)

    # hyper-exponential
    hyper.set_title("hyper-exponential")
    hyper.plot(xdata, ydata, label='data')
        # moments
    [l1, l2, p] = mm_Hyper[i]
    y = 1 - (p*np.exp(-l1*xlin) + (1-p)*np.exp(-l2*xlin))
    hyper.plot(xlin, y, label='moments' )
        # mle
    [l1, l2, p] = ml_Hyper[i]
    y = 1 - (p*np.exp(-l1*xlin) + (1-p)*np.exp(-l2*xlin))
    hyper.plot(xlin, y, label='maximum likelihood' )

    hyper.legend();
    
    # hypo-exponential
    hypo.set_title("hypo-exponential")
    hypo.plot(xdata, ydata, label='data')
        # moments
    [l1, l2] = mm_Hypo[i]
    y = 1 - l2*np.exp(-l1*xlin)/(l2-l1) + l1*np.exp(-l2*xlin)/(l2-l1)
    hypo.plot(xlin, y, label='moments' )
        # mle
    [l1, l2] = ml_Hypo[i]
    y = 1 - l2*np.exp(-l1*xlin)/(l2-l1) + l1*np.exp(-l2*xlin)/(l2-l1)
    hypo.plot(xlin, y, label='maximum likelihood' )

    hypo.legend();

    # exponential
    exp.set_title("exponential")
    exp.plot(xdata, ydata, label='data')
        # moments
    l = mm_Exp[i]
    y = 1 - np.exp(-l*xlin)
    exp.plot(xlin, y, label='moments')
        # mle
    [l] = ml_Exp[i]
    y = 1 - np.exp(-l*xlin)
    exp.plot(xlin, y, label='maximum likelihood')
    
    exp.legend()

    # uniform

    unif.set_title("uniform")
        # data
    unif.plot(xdata, ydata, label='data')
        # moments
    [a, b] = mm_Unif[i]
    # y = (xlin-a)/(b-a)
    y = []
    for x in xlin:
        if x<a:
            y.append(0)
        elif x>b:
            y.append(1)
        else:
            y.append((x-a)/(b-a))
    unif.plot(xlin, y, label='moments')

    unif.legend()

In [11]:
def gen_plot(i):
    
    fig, (mle,moments) = plt.subplots(1, 2)
    fig.suptitle('Dataset '+str(i+1))

    ydata = [*range(0,len(data[i]))]
    ydata = np.divide(ydata,len(data[i]))
    xdata = np.sort(data[i])

    xlin  = np.linspace(start=0, stop=50 if max(data[i])<50 else 190)

    
    moments.set_title("method of moments")
    moments.plot(xdata, ydata, label='data')
    #hyper
    [l1, l2, p] = mm_Hyper[i]
    y = 1 - (p*np.exp(-l1*xlin) + (1-p)*np.exp(-l2*xlin))
    moments.plot(xlin, y, label='hyper-exponential' )
    #hypo
    [l1, l2] = mm_Hypo[i]
    y = 1 - l2*np.exp(-l1*xlin)/(l2-l1) + l1*np.exp(-l2*xlin)/(l2-l1)
    moments.plot(xlin, y, label='hypo-exponential' )
    #exponential
    l = mm_Exp[i]
    y = 1 - np.exp(-l*xlin)
    moments.plot(xlin, y, label='exponential')
    #uniform
    [a, b] = mm_Unif[i]
    y = []
    for x in xlin:
        if x<a:
            y.append(0)
        elif x>b:
            y.append(1)
        else:
            y.append((x-a)/(b-a))
    moments.plot(xlin, y, label='uniform')
    moments.legend()
    
    
    mle.set_title("maximum likelihood method")
    mle.plot(xdata, ydata, label='data')
    #hyper
    [l1, l2, p] = ml_Hyper[i]
    print(l1, l2, p)
    y = 1 - (p*np.exp(-l1*xlin) + (1-p)*np.exp(-l2*xlin))
    mle.plot(xlin, y, label='hyper-exponential' )
    #hypo
    [l1, l2] = ml_Hypo[i]
    print(l1, l2)
    y = 1 - l2*np.exp(-l1*xlin)/(l2-l1) + l1*np.exp(-l2*xlin)/(l2-l1)
    mle.plot(xlin, y, label='hypo-exponential' )
    #exponential
    [l] = ml_Exp[i]
    print(l)
    y = 1 - np.exp(-l*xlin)
    mle.plot(xlin, y, label='exponential')
    mle.legend()

### Dataset 1

In [1]:
gen_plot(0)

NameError: name 'gen_plot' is not defined

### Dataset 2

In [13]:
gen_plot(1)

0.11475223610445998 0.18584158083844668 0.47459878698011204
0.14361621160965846 11380425.673679812
0.14361619701987782


### Dataset 3

In [14]:
gen_plot(2)

0.048727654171645055 0.2232534999822762 0.20553700350322907
0.1285900970884794 6834624.770236233
0.12859009951514075


### Dataset 4

In [15]:
gen_plot(3)

0.12928567330230845 0.1292856708056496 0.5020478914085292
0.18742270957801918 0.4167922325450846
0.12928567468731378


## Notes
As we can notice, the exponential computed with the method of moments and the exponential computed with the maximum likelihood method are the same on every dataset because the method of moments always reaches the best value for the parameter in that case.